# RailRat: A quick toolbox and Colab tutorial for using OpenStreetMap, OSMnx, & NetworkX for RailWay routing and isochrone analysis. 

# Mount your Google Drive before following the notebook. 

In [1]:
#!pip install osmnx # not standard in colab

In [2]:
import sys
# save the RailRat.py module to your directory of choice, add the directory below:
sys.path.insert(1, '/content/drive/MyDrive/Colab_Notebooks/Route_Engine/RailRat') 
import RailRat
import osmnx as ox
ox.config(use_cache=False)
import networkx as nx
import folium
from matplotlib import cm
import matplotlib.colors as mpl_colors

# 1) Create railway graph from raw OSM data and save to a .graphml

* ## Use the following overpass-turbo query to get railway data from Pheonix, AZ: https://overpass-turbo.eu/s/1eKp

* ## Make sure line #6 of the query reads `[out:xml]` and not `[out:json]`

* ## Once the query completes, click "Export" and download the the data as raw OSM data (should be XML format). 

* ## Save the .osm file to your drive as phoenix_rail_map.osm

* ## Then execute the following cells

In [3]:
# Load the raw osm data to a graph and plot with folium
R = ox.graph.graph_from_xml(filepath='/content/drive/MyDrive/Colab_Notebooks/Route_Engine/RailRat/phoenix_rail_map.osm') # converts the raw OSM railway data to a Networkx graph
ox.folium.plot_graph_folium(R,color='gray',graph_map=folium.Map(width=500,height=500))

Output hidden; open in https://colab.research.google.com to view.

In [4]:
# set the rail speeds and edge travel times
nx.set_edge_attributes(R, values=72, name="speed_kph")
ox.speed.add_edge_travel_times(R, precision=2)
edges = list(R.edges())
R.get_edge_data(edges[0][0],edges[0][1]) # print an edge to see the attributes you added

{0: {'geometry': <shapely.geometry.linestring.LineString at 0x7faf51953350>,
  'length': 8069.648000000002,
  'name': 'Phoenix Subdivision',
  'oneway': False,
  'osmid': 645837878,
  'ref': '7208',
  'speed_kph': 72,
  'travel_time': 403.48}}

In [5]:
# save the graph to a .graphml file
ox.io.save_graphml(R, filepath='/content/drive/MyDrive/Colab_Notebooks/Route Engine/phoenix_rail.graphml')

# 2) Load the RailWay graph and use the RailRat module for routing and isochrone analysis

In [6]:
# Load the .graphml and plot the graph again
R = ox.io.load_graphml(filepath='/content/drive/MyDrive/Colab_Notebooks/Route Engine/phoenix_rail.graphml') # load the graph
map = folium.Map(width=500,height=500)
ox.folium.plot_graph_folium(R,graph_map=map,color='gray')

Output hidden; open in https://colab.research.google.com to view.

# 2a) RailRat Routing: One origin to many destinations

In [7]:
start = 'Phoenix'
destinations = ['Avondale, AZ','Glendale, AZ','Tempe, AZ']
routes, df = RailRat.shortest_path_one_to_many(R,start=start,destinations=destinations,start_datetime=(2015,3,13,5,12,16))
df

,Origin,Destintation,Arrival-Time(UTC),Distance(m),Travel-Time(s)
0,Phoenix,"Avondale, AZ",2015-03-13 05:33:43.030000+00:00,25740.089,1287.03
1,Phoenix,"Glendale, AZ",2015-03-13 05:25:40.670000+00:00,16094.130,804.67
2,Phoenix,"Tempe, AZ",2015-03-13 05:23:45.870000+00:00,13797.386,689.87


In [8]:
# plot the routes on a folium map
map = folium.Map(width=500,height=500)
ox.folium.plot_graph_folium(R,graph_map=map,color='gray')
cmap = cm.get_cmap('rainbow', len(routes))
for r in range(len(routes)):
  rgba = cmap(r)
  ox.folium.plot_route_folium(R, routes[r], route_map=map, color=mpl_colors.rgb2hex(rgba),weight=7,fit_bounds=False)
map

Output hidden; open in https://colab.research.google.com to view.

# 2b) RailRat Isochrones: Arrival times at all possible node locations

In [9]:
df,map = RailRat.rail_isochrone(G=R,start='Phoenix',show_map=True)
df

,osmid,Coordinates,Travel-Time(s),Arrival-Time(UTC)
0,329598861,"{'y': 33.4442466, 'x': -112.0745422}",0.00,2022-01-07 18:00:42.547703+00:00
1,1450708764,"{'y': 33.4440799, 'x': -112.0769289}",11.15,2022-01-07 18:00:53.697703+00:00
2,1450708700,"{'y': 33.4440013, 'x': -112.0783508}",17.79,2022-01-07 18:01:00.337703+00:00
3,42556884,"{'y': 33.444201, 'x': -112.0679495}",30.59,2022-01-07 18:01:13.137703+00:00
4,1819838360,"{'y': 33.444229, 'x': -112.066159}",38.89,2022-01-07 18:01:21.437703+00:00
...,...,...,...,...
1228,3372779838,"{'y': 33.2262559, 'x': -111.8373731}",2297.87,2022-01-07 18:39:00.417703+00:00
1229,3372779843,"{'y': 33.2268237, 'x': -111.8378704}",2301.78,2022-01-07 18:39:04.327703+00:00
1230,3372779850,"{'y': 33.2273325, 'x': -111.837655}",2304.08,2022-01-07 18:39:06.627703+00:00
1231,3372779826,"{'y': 33.2216519, 'x': -111.8366443}",2309.09,2022-01-07 18:39:11.637703+00:00


In [10]:
# Show the isochrone folium map
map

Output hidden; open in https://colab.research.google.com to view.

# The end!